# Evaluating the Performance of our MER System

Our system uses the audio features of a song to predict the empirical PDF of a song in the valence/arousal space. We'll evaluate how accurately we are able to predict these PDFs by comparing them to the actual PDFs which were obtained using kernel density estimation of point-level valence/arousal data. 

In [33]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from mer.learn_kde_audio import map_factor_learn, emotion_space_map

# Read in Data

1. Audio Feature Data
2. Empirical PDFs

In [35]:
data_path = Path.cwd().parent / 'data' / 'final'

audio_df = pd.read_csv(data_path / 'audio_feature_71_train.csv', index_col=0) \
    .sort_index() \
    .values
pdf_df = pd.read_csv(data_path / 'Time_Average_Gamma_0_1.csv', index_col='song_id') \
    .sort_index() \
    .values

In [36]:
audio_df.shape

(744, 71)

In [37]:
pdf_df.shape

(744, 256)

# Evaluate Performance with Cross-Validation

We'll use 5-fold cross validation and our performance metric will be the coefficient of determination $R^2$

In [30]:
def r_squared(x, y):
    """returns the coefficient of determination"""
    return np.corrcoef(x, y)[0, 1] ** 2

## One-Hold-Out Cross Validation

This method was not used in the paper

In [58]:
# one-hold-out cross-validation

r2_vals = []
for i in range(len(audio_df)):
    audio_test, pdf_test = audio_df[i, :], pdf_df[i, :]
    audio_train, pdf_train = np.delete(audio_df, i, 0), np.delete(pdf_df, i, 0)
    
    map_factors = map_factor_learn(audio_train, audio_test)
#     print(audio_train.shape)
#     print(map_factors.shape)
#     print(pdf_train.shape)
    pred_pdf = emotion_space_map(pdf_train, map_factors)
    
    r2_vals.append(r_squared(pdf_test, pred_pdf))

In [73]:
print('Mean R^2: {}\n\
Standard Deviation R^2: {}'.format(np.mean(r2_vals), np.std(r2_vals)))

Mean R^2: 0.7494501060727498
Standard Deviation R^2: 0.22781594703318286


## 5-Fold Cross Validation 

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [64]:
r2_5fold = []
for train_index, test_index in kf.split(audio_df):
    audio_train, audio_test = audio_df[train_index], audio_df[test_index]
    pdf_train, pdf_test = pdf_df[train_index], pdf_df[test_index]
    for song_audio in audio_test:
        map_factors = map_factor_learn(audio_train, song_audio)
        new_pdf = emotion_space_map(pdf_train, map_factors)
        r2_5fold.append(r_squared(pdf_test, pred_pdf))
    
    

In [74]:
print('Mean R^2: {}\n\
Standard Deviation R^2: {}'.format(np.mean(r2_5fold), np.std(r2_5fold)))

Mean R^2: 0.3778321825343548
Standard Deviation R^2: 0.33401046758544595
